In [20]:
from scipy.io.arff import loadarff
import pandas as pd

#### Lectura de fichero

In [21]:
df_sick_train, meta_train = loadarff('sick/sick.fold.000000.train.arff')
df_sick_test, meta_test = loadarff('sick/sick.fold.000000.train.arff')

In [22]:
df_sick_train = pd.DataFrame(df_sick_train)
df_sick_test = pd.DataFrame(df_sick_test)
df_sick_train = df_sick_train.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df_sick_test = df_sick_test.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

C:\Users\xavid\AppData\Local\Temp\ipykernel_17396\2591950523.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_sick_train = df_sick_train.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
C:\Users\xavid\AppData\Local\Temp\ipykernel_17396\2591950523.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_sick_test = df_sick_test.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


#### Data analysis

In [4]:
df_sick_train[df_sick_train['age'].isna()]

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
1950,NaN,F,t,f,f,f,f,f,t,f,...,t,120.0,t,0.82,t,146.0,f,NaN,other,negative


- sex tiene 3 categorías
- las 2 de TBG fuera
- NaNs: regresión lineal
- Age quitar la fila con nan en Edad

In [18]:
for col in df_sick_train.columns:
    print(f'{col}: {df_sick_train[col].unique()}')

age: [0.07929515 0.14317181 0.17180617 0.16740088 0.13436123 0.12114537
 0.17621145 0.11674009 0.16960352 0.07268722 0.15638767 0.15198238
 0.16299559 0.04185022 0.05947137 0.11453744 0.07488987 0.15859031
 0.03744493 0.08370044 0.1123348  0.04845815 0.0969163  0.06387665
 0.07048458 0.13215859 0.06167401 0.06828194 0.1784141  0.10572687
 0.12334802 0.1277533  0.09251101 0.11013216 0.1938326  0.16079295
 0.14757709 0.04405286 0.02202643 0.0660793  0.17400881 0.13876652
 0.14096916 0.08590308 0.09030837 0.09471366 0.01321586 0.16519824
 0.11894273 0.13656388 0.08810573 0.0814978  0.10792952 0.14537445
 0.05286344 0.05066079 0.14977974 0.04625551 0.07709251 0.03964758
 0.12555066 0.10352423 0.12995595 0.03524229 0.09911894 0.18061674
 0.15418502 0.05506608 0.18942731 0.030837   0.05726872 0.00881057
 0.02863436 0.10132159 0.02643172 0.19603524 0.0154185  0.19823789
 0.         0.18281938 0.01101322 0.18502203 0.03303965 0.19162996
 0.18722467 0.01982379 0.02422907 0.00220264 0.00660793 0

In [6]:
columns_with_nans = []
for col in df_sick_train.columns:
    sum_nans = sum(df_sick_train[col].isna())
    if sum_nans / len(df_sick_train) * 100> 0.5:
        print(f'{col}: {sum_nans}')
        columns_with_nans.append(col)

TSH: 331
T3: 689
TT4: 209
T4U: 356
FTI: 354
TBG: 3396


In [7]:
df_sick_test.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,37.0,F,t,f,f,f,f,f,f,f,...,t,156.0,t,0.84,t,187.0,f,NaN,other,negative
1,37.0,M,f,f,f,f,f,f,f,f,...,t,68.0,t,0.94,t,72.0,f,NaN,other,negative
2,66.0,F,t,f,f,f,f,f,f,f,...,t,120.0,t,0.91,t,132.0,f,NaN,other,negative
3,79.0,F,f,f,f,f,f,f,f,f,...,t,94.0,t,0.86,t,109.0,f,NaN,other,sick
4,79.0,F,f,f,f,f,f,f,f,f,...,t,91.0,t,0.99,t,91.0,f,NaN,other,negative


#### Drop rows age

#### Min-max scaler

In [8]:
from utils import min_max_scaler

df_sick_train, df_sick_test = min_max_scaler(df_sick_train,df_sick_test, ['age'])

C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[numerical_cols] = scaler.transform(df_train[numerical_cols])
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[numerical_cols] = scaler.transform(df_test[numerical_cols])


In [16]:
from utils import one_hot_encoding

df_sick_train, df_sick_test = one_hot_encoding(df_sick_train,df_sick_test)

In [10]:
from utils import binary_encoding
df_sick_train, df_sick_test = binary_encoding(df_sick_train,df_sick_test)

C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:65: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train[binary_features] = df_train[binary_features].replace({'t': 1, 'f': 0})
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[binary_features] = df_test[binary_features].replace({'t': 1, 'f': 0})


In [23]:
df_sick_train['Class'].nunique()

2

In [11]:
from utils import fill_nans
df_sick_train, df_sick_test = fill_nans(df_sick_train, df_sick_test, columns_with_nans)

ValueError: could not convert string to float: 'negative'